#### 1. Extract DATE and TIME from payment_timestamp and create new columns payment_date and payment_time.

#### 2. Map payment status to descriptive values
#####  (1 -> Success, 2 -> Pending, 3 -> Cancelled, 4 -> Failed).

#### 3. Write transformed data to Silver schema. 



In [0]:
df_payments_raw = spark.sql('SELECT * FROM gizmobox.bronze.py_payments')
display(df_payments_raw)

#### 1,2 Extract DATE and TIME from payment_timestamp and create new columns payment_date and payment_time.
#### Map payment status to descriptive values (1 -> Success, 2 -> Pending, 3 -> Cancelled, 4 -> Failed). 


In [0]:
from pyspark.sql import functions as F
df_payments_transformed = (
                df_payments_raw.select(
                                        "payment_id",
                                        'order_id',
                                        F.date_format("payment_timestamp", "yyyy-MM-dd").alias("payment_date"),
                                        F.date_format("payment_timestamp", "HH:MM:SS").alias("payment_time"),
                                        F.when(df_payments_raw.psyment_status == 1, "Success")
                                         .when(df_payments_raw.psyment_status == 2, "Pending")
                                         .when(df_payments_raw.psyment_status == 3, "Cancelled")
                                         .when(df_payments_raw.psyment_status == 4, "Failed").alias('payment_status'),
                                        'payment_method'
                                      )
                )
display(df_payments_transformed)

### 3. Write transformed data to Silver schema.

In [0]:
df_payments_transformed.writeTo("gizmobox.silver.py_payments").createOrReplace()

In [0]:
%sql
SELECT * FROM gizmobox.silver.py_payments;